In [41]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import pingouin as pg
import plotly.graph_objects as go
from os.path import join as pjoin

sys.path.append("../../")
import circletrack_behavior as ctb
import plotting_functions as pf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
## Settings
parent_dir = 'MultiCon_Imaging'
experiment_dir = 'MultiCon_Imaging4'
lin_path = f'../../../{parent_dir}/{experiment_dir}/output/lin_behav/'
circle_path = f'../../../{parent_dir}/{experiment_dir}/output/behav/'
fig_path = f'../../../{parent_dir}/{experiment_dir}/intermediate_figures'
mouse_list = ['mc42', 'mc43']
chance_color = 'darkgrey'
avg_color = 'midnightblue'
subject_color = 'darkgrey'

### Linear track: rewards across days.

In [43]:
lin_dict = {'mouse': [], 'day': [], 'rewards': []}
for mouse in os.listdir(lin_path):
    mouse_path = pjoin(lin_path, mouse)
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        lin_dict['mouse'].append(mouse)
        lin_dict['day'].append(idx+1)
        lin_dict['rewards'].append(np.sum(behav['water']))
lin_df = pd.DataFrame(lin_dict)

In [49]:
## Plot rewards across days on linear track
fig = pf.plot_behavior_across_days(lin_df, x_var='day', y_var='rewards', groupby_var=['day'], plot_transitions=None,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=False,
                                   x_title='Day', y_title='Rewards', titles=['Linear Track'], height=500, width=500)
fig.show()
fig.write_image(pjoin(fig_path, 'linear_track_rewards.png'))

In [45]:
## Linear track x position across time
## Rewards are red circles, licks are sheer black circles.
fig = pf.custom_graph_template(x_title='Time (s)', y_title='X Position')
fig.add_trace(go.Scatter(x=behav['t'], y=behav['x'], mode='lines', line_color='darkgrey', showlegend=False))
fig.add_trace(go.Scatter(x=behav['t'][behav['water']], y=behav['x'][behav['water']], mode='markers', marker_color='red', showlegend=False))
fig.add_trace(go.Scatter(x=behav['t'][behav['lick_port'] != -1], y=behav['x'][behav['lick_port'] != -1], mode='markers', 
                         marker_color='black', showlegend=False, opacity=0.6))
fig.show()

### Circle track: 1st/5th lick accuracy and rewards across days.

In [ ]:
circletrack_results = {'mouse': [], 'day': [], 'session': [], 'lick_accuracy_thresh1': [], 'lick_accuracy_thresh5': [], 'rewards': []}
for mouse in os.listdir(circle_path)[:-1]:
    mouse_path = pjoin(circle_path, mouse)
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav_data = behav[~behav['probe']]
        reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
        pc_thresh1 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=1, by_trials=False)
        pc_thresh5 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=5, by_trials=False)
        circletrack_results['mouse'].append(mouse)
        circletrack_results['day'].append(idx+1)
        circletrack_results['session'].append(np.unique(behav['session'])[0])
        circletrack_results['lick_accuracy_thresh1'].append(pc_thresh1)
        circletrack_results['lick_accuracy_thresh5'].append(pc_thresh5)
        circletrack_results['rewards'].append(np.sum(behav_data['water']))
ct_df = pd.DataFrame(circletrack_results)

In [ ]:
## Plot 5th lick accuracy across days
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy_thresh5', groupby_var=['day'], plot_transitions=None,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=True,
                                   x_title='Day', y_title='5th Lick Accuracy', titles=['Circle Track'], height=500, width=500)
fig.show()

In [ ]:
## Plot 1st lick accuracy across days
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy_thresh1', groupby_var=['day'], plot_transitions=None,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=True,
                                   x_title='Day', y_title='1st Lick Accuracy', titles=['Circle Track'], height=500, width=500)
fig.show()

In [ ]:
## Plot rewards across days
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='rewards', groupby_var=['day'], plot_transitions=None,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=True,
                                   x_title='Day', y_title='rewards', titles=['Circle Track'], height=500, width=500)
fig.show()